## <center> Costum environment creation with OpenAI GYM </center>

In the following notebook a costum environment will be created, in which an agent will have to adjust the temperature of a shower. During this, the agenet will have to deal with temperature fluctuations and a certain time limit.

### 1. Creating the environment

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [2]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp and start time
        self.reset()
        
    def step(self, action):

        # Apply action
        self.state += action - 1
        # Decrease temperature action = 0 --> T + 0 - 1 = T - 1
        # Stall temperature action = 1 --> T + 1 - 1 = T + 0
        # Increase temperature action = 2 --> T + 2 -1 = T + 1
        
        
        # Apply temperature noise --> more realistic model
        self.state += random.randint(-1,1)

        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if 37 <= self.state <= 39: 
            reward = 1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement visualization --> in this case is not built
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state


In [3]:
env = ShowerEnv()
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    temps = []
    
    while not done:
        action = env.action_space.sample()
        temp, reward, done, info = env.step(action)
        temps.append(temp) 
        score +=reward
    mean_temp = np.mean(np.array(temps))
    print(f'Episode: {episode}, Mean temperature: {mean_temp:.2f} Score: {score}')

Episode: 1, Mean temperature: 39.20 Score: -44
Episode: 2, Mean temperature: 38.73 Score: -30
Episode: 3, Mean temperature: 33.67 Score: -34
Episode: 4, Mean temperature: 28.75 Score: -60
Episode: 5, Mean temperature: 44.00 Score: -48
Episode: 6, Mean temperature: 40.83 Score: -48
Episode: 7, Mean temperature: 40.28 Score: -26
Episode: 8, Mean temperature: 47.65 Score: -50
Episode: 9, Mean temperature: 39.90 Score: -28
Episode: 10, Mean temperature: 38.80 Score: -46


/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


### 2. Creating a Deep Learning Model Agent with Keras-RL

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [5]:
num_actions = env.action_space.n
num_observations = env.observation_space.shape[0]
print(f"There are {num_actions} possible actions and {num_observations} observations")

There are 3 possible actions and 1 observations


In [6]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(128, activation='relu', input_shape=(states, )))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(18, activation='relu'))
    model.add(Dense(9, activation='relu'))

    model.add(Dense(actions, activation='linear'))

    return model

In [7]:
model = build_model(num_observations, num_actions)

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               256       
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 18)                594       
                                                                 
 dense_4 (Dense)             (None, 9)                 171       
                                                                 
 dense_5 (Dense)             (None, 3)                 30        
                                                                 
Total params: 11,387
Trainable params: 11,387
Non-traina

In [8]:
memory = SequentialMemory(limit=20000, window_length=1)
policy = LinearAnnealedPolicy(BoltzmannQPolicy(),
                              attr='tau', 
                              value_max=1.0, 
                              value_min=0.001,  
                              value_test=0.0005, 
                              nb_steps=400000) 
dqn = DQNAgent(model=model, 
               nb_actions=num_actions, 
               memory=memory, 
               nb_steps_warmup=10,
               target_model_update=1e-2, 
               policy=policy)

# Compilation
dqn.compile(Adam(learning_rate=0.0001), metrics=['mae']) 

In [9]:
dqn.fit(env, nb_steps=400000, visualize=False, verbose=1)

Training for 400000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 22:31 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   20/10000 [..............................] - ETA: 15:20 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs

   40/10000 [..............................] - ETA: 8:25 - reward: -0.6500

/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 23 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 24 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 25 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 26 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 27 + 1) instead
  batch_idxs

10000/10000 [==============================] - 95s 9ms/step - reward: -0.6916
166 episodes - episode_reward: -41.518 [-60.000, 12.000] - loss: 0.983 - mae: 6.615 - mean_q: -9.267 - mean_tau: 0.988

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 93s 9ms/step - reward: -0.7450
167 episodes - episode_reward: -44.754 [-60.000, 12.000] - loss: 1.475 - mae: 9.593 - mean_q: -13.622 - mean_tau: 0.963

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 94s 9ms/step - reward: -0.7272
167 episodes - episode_reward: -43.545 [-60.000, 12.000] - loss: 1.585 - mae: 10.449 - mean_q: -14.902 - mean_tau: 0.938

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 93s 9ms/step - reward: -0.6760
166 episodes - episode_reward: -40.687 [-60.000, 18.000] - loss: 2.275 - mae: 12.110 - mean_q: -17.435 - mean_tau: 0.913

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 94s 9ms/step - r

In [10]:
results = dqn.test(env, nb_episodes=150, visualize=False)
print(np.mean(results.history['episode_reward']))

Testing for 150 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 54.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 58.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 56.000, steps: 60
Episode 9: reward: 58.000, steps: 60
Episode 10: reward: 58.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 58.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 56.000, steps: 60
Episode 22: reward: 58.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, st